In [26]:
import re
import pandas as pd
import pdfplumber
from collections import namedtuple

In [27]:
Line = namedtuple('Line','Name Element Info BaseValue GOE J1 J2 J3 J4 J5 J6 J7 J8 J9 ScoresOfPanel')
# names of column headers for dataframe

In [28]:
tes = re.compile(r' [+-]?\d{1}\ [+-]?\d{1}\ [+-]?\d{1}')
# recognize lines containing technical element scores

In [29]:
tes.search(' -0.08 -1 0 0 -1 0 1 -1 1 0')
# check if tes.compile works

<re.Match object; span=(6, 13), match=' -1 0 0'>

In [30]:
pcs = re.compile(r'[+-]?\d+(?:\.\d+)?\ [+-]?\d+(?:\.\d+)?\ [+-]?\d+(?:\.\d+)?')
# recognize lines containing presentation component scores

In [31]:
file = 'Freeskate.pdf'

In [23]:
lines = []

with pdfplumber.open(file) as pdf:
    pages = pdf.pages
    for page in pdf.pages:
        text = page.extract_text()       # get text from freeskate file
        for line in text.split('\n'):     # split by line
            tes1 = tes.search(line)        # sort lines into name, tcs, or pcs
            name1 = re.match(r'\d{1,2}\ (?<= )\w+\ (?<= )\w+',line)
            pcs1 = pcs.search(line)
            if tes1:
               items = line.split()
               items = items[1:]      # remove leading number indicating order of elements skated 
               if 'x' in items:       # x represents jumps with higher base value due to placement 
                   items.remove('x')   #  in second half of program - can be recognized later in dataframe
               if len(items) == 13:   
                   items.insert(1,' ')   # if there is nothing in the info column, need to add empty entry to maintain spacing
               if y != None:             # add the name into the list 
                   items.insert(0,y)
                   lines.append(Line(*items))
               print(len(items), items)
            elif name1:
                if name1.group(0) != None:  
                    y = name1.group(0)     
                    y = y[2:]             # remove rank and whitespace before name
                    print(y)
            elif pcs1:
               items1 = line.split()
               if 'Skating' not in items1:  # Skating skills needs to be handled separately - pcs lines will not have any info,
                   items1.insert(1,' ')     # so info column space needs to be added same as above
                   items1.insert(1,' ')     # second blank is added since there is no base value in component scores
                   #print(len(items1), items1)
               elif 'Skating' in items1:
                   items1 = items1[2:]
                   items1.insert(0,'Skating Skills')   # Skating Skills is two words but needs to end up as a single entry
                   items1.insert(1,' ')          # - "skating","skills" -> "Skating Skills"
                   items1.insert(1,' ')
               if y != None:
                   items1.insert(0,y)
                   lines.append(Line(*items1))
                   print(len(items1), items1)
                
            




Loena HENDRICKX
15 ['Loena HENDRICKX', '3Lz+3Tq', 'q', '10.10', '-0.08', '-1', '0', '0', '-1', '0', '1', '-1', '1', '0', '10.02']
15 ['Loena HENDRICKX', '2A', ' ', '3.30', '0.75', '3', '2', '2', '2', '2', '2', '2', '3', '3', '4.05']
15 ['Loena HENDRICKX', '3F', ' ', '5.30', '1.44', '3', '3', '3', '2', '3', '3', '3', '2', '2', '6.74']
15 ['Loena HENDRICKX', '2A', ' ', '3.30', '0.85', '3', '2', '3', '2', '2', '3', '3', '2', '3', '4.15']
15 ['Loena HENDRICKX', 'CCoSp4', ' ', '3.50', '1.20', '4', '2', '3', '4', '3', '4', '3', '3', '4', '4.70']
15 ['Loena HENDRICKX', 'ChSq1', ' ', '3.00', '1.50', '3', '3', '3', '3', '3', '3', '3', '2', '4', '4.50']
15 ['Loena HENDRICKX', '3Lz+2T', ' ', '7.92', '1.35', '3', '2', '3', '2', '2', '2', '2', '2', '3', '9.27']
15 ['Loena HENDRICKX', '3F+2T+2Lo', ' ', '9.13', '0.68', '1', '1', '2', '1', '1', '2', '1', '3', '1', '9.81']
15 ['Loena HENDRICKX', '3S', ' ', '4.73', '0.92', '2', '1', '3', '2', '2', '3', '2', '2', '2', '5.65']
15 ['Loena HENDRICKX', 'FCCo

In [33]:
df = pd.DataFrame(lines)
df.head(15)

,Name,Element,Info,BaseValue,GOE,J1,J2,J3,J4,J5,J6,J7,J8,J9,ScoresOfPanel
0,Loena HENDRICKX,3Lz+3Tq,q,10.10,-0.08,-1,0,0,-1,0,1,-1,1,0,10.02
1,Loena HENDRICKX,2A,,3.30,0.75,3,2,2,2,2,2,2,3,3,4.05
2,Loena HENDRICKX,3F,,5.30,1.44,3,3,3,2,3,3,3,2,2,6.74
3,Loena HENDRICKX,2A,,3.30,0.85,3,2,3,2,2,3,3,2,3,4.15
4,Loena HENDRICKX,CCoSp4,,3.50,1.20,4,2,3,4,3,4,3,3,4,4.70
5,Loena HENDRICKX,ChSq1,,3.00,1.50,3,3,3,3,3,3,3,2,4,4.50
6,Loena HENDRICKX,3Lz+2T,,7.92,1.35,3,2,3,2,2,2,2,2,3,9.27
7,Loena HENDRICKX,3F+2T+2Lo,,9.13,0.68,1,1,2,1,1,2,1,3,1,9.81
8,Loena HENDRICKX,3S,,4.73,0.92,2,1,3,2,2,3,2,2,2,5.65
9,Loena HENDRICKX,FCCoSp4,,3.50,1.05,3,2,4,3,3,3,3,4,2,4.55
